In [ ]:
!pip install mercantile
!pip install mapbox_vector_tile
!pip install vt2geojson
!pip install osmnx

In [2]:
import os
from google.colab import drive
import osmnx as ox
import networkx as nx
import pickle
import json
import time
import requests
import mercantile, mapbox_vector_tile
from vt2geojson.tools import vt_bytes_to_geojson


# Mount to Gdrive


In [ ]:
# connect to drive
drive.mount('/gdrive')

# Setup

In [ ]:
# load mapillary access token
file_path = '/gdrive/MyDrive/mapillary_token.txt'
with open(file_path, 'r') as file:
    access_token = file.read()

# define path
path = '/gdrive/MyDrive/berlin_bike_CV/'
folder = 'metadata'
mapillary_folder = 'mapillary'

name = 'DSR'

# Download and Save Graph of Berlin

In [3]:
### define area and network type

# graph of Berlin
# place_name = "Berlin, Germany"
# G = ox.graph_from_place(place_name, network_type="bike")
# point: [lat, lon]
# place_dsr = [52.500632881796925, 13.334863741376868]
# G = ox.graph_from_point(place_dsr, dist=1000, network_type="bike")
# [north, south, east, west]
bb_dsr = [52.534947724864615, 52.48412181578994, 13.379150390625, 13.295654296875]
network_type = 'bike'

G = ox.graph_from_bbox(*bb_dsr, network_type=network_type)

In [4]:
# find bounding box
gdf = ox.graph_to_gdfs(G, nodes=False, edges=True)
bounding_box = gdf.total_bounds

# show bounding box coordinates
print("Bounding Box:")
print("west: ", bounding_box[0])
print("south:", bounding_box[1])
print("east: ", bounding_box[2])
print("north:", bounding_box[3])

Bounding Box:
west: 13.2956688
south: 52.4839151
east: 13.3791321
north: 52.5349833


In [ ]:
# save original graph
graph_name = '_graph_origin.pkl'
graph_path = os.path.join(path, folder, name + graph_name)
with open(graph_path, 'wb') as pickle_file:
    pickle.dump(G, pickle_file)

# Find Set of Tiles for Berlin

In [8]:
# vector tile endpoints
tile_coverage = "mly1_public"

# vector tile endpoint
tile_layer = "image"

# get the list of tiles intersecting the bounding box
tiles = list(mercantile.tiles(*bounding_box, 14))

# Divide and Save Set of Edges of Graph per Tile

In [ ]:
def create_edges():
  edges = []

  for u, v, k in G.edges(keys=True):
          edges.append((u, v, k))
  return edges

In [ ]:
def extract_edges(tile, remaining_edges):

    edges = {}
    remove_edges = []
    min_lon, min_lat, max_lon, max_lat = mercantile.bounds(tile)

    for (u, v, k) in remaining_edges:
        u_x, u_y = G.nodes[u]['x'], G.nodes[u]['y']
        v_x, v_y = G.nodes[v]['x'], G.nodes[v]['y']

        if (min_lon <= u_x <= max_lon and min_lat <= u_y <= max_lat) or (min_lon <= v_x <= max_lon and min_lat <= v_y <= max_lat):
          edges[(u, v, k)] = []
          remove_edges.append((u, v, k))

    for edge in reversed(remove_edges):
        remaining_edges.remove(edge)

    return edges, remaining_edges

In [ ]:
def assign_edges_to_tiles(tiles):
    remaining_edges = create_edges()
    # print(len(remaining_edges))
    tiles_data = {}

    for i, tile in enumerate(tiles):
        edges , remaining_edges = extract_edges(tile, remaining_edges)
        if edges:
          tiles_data[mercantile.quadkey(tile)] = edges

    return tiles_data

In [ ]:
# assign edgeset per tile
tiles_data = assign_edges_to_tiles(tiles)

In [ ]:
# save tiles with dict of edges
file_name = '_tiles_edges.pkl'
file_path = os.path.join(path, folder, name + file_name)
with open(file_path, 'wb') as pickle_file:
    pickle.dump(tiles_data, pickle_file)

# Download and Save Mapillary Metadata per Tile

In [ ]:
# TODO: take Mapillary API downloading rates into acount
sleep_rate = 0.5

west, south, east, north = bounding_box

# loop over list of tiles to get Mapillary endpoints data and make request
for tile_quadkey in tiles_data:
    time.sleep(sleep_rate)
    tile = mercantile.quadkey_to_tile(tile_quadkey)
    tile_url = 'https://tiles.mapillary.com/maps/vtp/{}/2/{}/{}/{}?access_token={}'.format(tile_coverage,tile.z,tile.x,tile.y,access_token)
    response = requests.get(tile_url)
    data = vt_bytes_to_geojson(response.content, tile.x, tile.y, tile.z,layer=tile_layer)

    tile_data = { "type": "FeatureCollection", "features": [] }

    for feature in data['features']:

        # avoid panorama pictures
        if feature["properties"]["is_pano"]:
            continue

        # get coordinates
        lng = feature["geometry"]["coordinates"][0]
        lat = feature["geometry"]["coordinates"][1]

        # ensure feature falls inside bounding box since tiles can extend beyond
        if lng > west and lng < east and lat > south and lat < north:

            tile_data['features'].append(feature)

    # save data per tile
    file_name = tile_quadkey + '.geojson'
    file_path = os.path.join(path, folder, mapillary_folder, file_name)
    with open(file_path, 'w') as f:
        json.dump(tile_data, f)


print('Done')